In [1]:
!pip install pkbar
!pip install shapely

In [2]:
import os
import json
import argparse
import torch
import random
import numpy as np
from pathlib import Path
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
# # from model.SegnetEnc_FftDec import SegnetEnc
# from model.Efficientnet import EfficientNetEnc
from model.DeneNet_FFT import Dense_fft

from dataset.dataset import RADIal
from dataset.encoder import ra_encoder
from dataset.dataloader import CreateDataLoaders
import pkbar
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from loss import pixor_loss
from utils.evaluation import run_evaluation
import torch.nn as nn
import torch
torch.cuda.empty_cache()


/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# !pip install efficientnet_pytorch

In [4]:
config_file_path = "config/config_FFTRadNet_192_56.json"

# Read the content of the JSON file and assign it to the config variable
with open(config_file_path, 'r') as f:
    config = json.load(f)
resume_path = None  # Assuming you don't have a resume path for now

print("DetectionHead",config['model']['DetectionHead'])
print("SegmentationHead",config['model']['SegmentationHead'])
# Setup random seed


DetectionHead True
SegmentationHead True


In [5]:

torch.manual_seed(config['seed'])
np.random.seed(config['seed'])
random.seed(config['seed'])
torch.cuda.manual_seed(config['seed'])

In [6]:

# create experience name
curr_date = datetime.now()
exp_name = config['name'] + '___' + curr_date.strftime('%b-%d-%Y___%H:%M:%S')
# print(exp_name)

# Create directory structure
# output_folder = Path(config['output']['dir'])
output_folder = Path("DeneNet_FFT_D&S_rand")

output_folder.mkdir(parents=True, exist_ok=True)
(output_folder / exp_name).mkdir(parents=True, exist_ok=True)
# and copy the config file
with open(output_folder / exp_name / 'config.json', 'w') as outfile:
    json.dump(config, outfile)


In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


In [8]:


# set device
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Initialize tensorboard
writer = SummaryWriter(output_folder / exp_name)

# Load the dataset
enc = ra_encoder(geometry = config['dataset']['geometry'], 
                    statistics = config['dataset']['statistics'],
                    regression_layer = 2)


In [9]:
dataset = RADIal(root_dir = config['dataset']['root_dir'],
                    statistics= config['dataset']['statistics'],
                    encoder=enc.encode,
                    difficult=True)

train_loader, val_loader, test_loader = CreateDataLoaders(dataset,config['dataloader'],config['seed'])
print(train_loader)

===========  Dataset  ==================:
      Mode: random
      Train Val ratio: [0.7, 0.15, 0.15]
      Training: 5775  indexes... [6525, 4299, 7696]
      Validation: 1237  indexes... [5270, 811, 648]
      Test: 1239  indexes... [713, 6650, 4297]



In [10]:
# Create the model
# net = FFTRadNet(blocks = config['model']['backbone_block'],
#                     mimo_layer  = config['model']['MIMO_output'],
#                     channels = config['model']['channels'], 
#                     regression_layer = 1, 
#                     detection_head = "True", 
#                     segmentation_head = "True")

# # net = Unet_fft(detection_head = config['model']['DetectionHead'],
# #                         segmentation_head = config['model']['SegmentationHead'])
# net = EfficientNetEnc (n_channels=32, n_classes=1, detection_head=True, segmentation_head=True)
net = Dense_fft(depth=121, growth_rate=32, reduction=0.5, bottleneck=True, dropRate=0.0,detection_head=True,segmentation_head=True)

net.to('cuda')
# print("DetectionHead",config['model']['DetectionHead'])
# print("SegmentationHead",config['model']['SegmentationHead'])

Dense_fft(
  (densenet_encoder): DenseNetEncoder(
    (conv1): Conv2d(192, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (block1): DenseBlock(
      (layer): Sequential(
        (0): BottleneckBlock(
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (1): BottleneckBlock(
          (bn1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv1): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         

In [11]:
# Optimizer
lr = float(config['optimizer']['lr'])
step_size = int(config['lr_scheduler']['step_size'])
gamma = float(config['lr_scheduler']['gamma'])
optimizer = optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

num_epochs=int(config['num_epochs'])


print('===========  Optimizer  ==================:')
print('      LR:', lr)
print('      step_size:', step_size)
print('      gamma:', gamma)
print('      num_epochs:', num_epochs)
print('')

# Train
startEpoch = 0
global_step = 0
##### 

history = {'train_loss':[],'val_loss':[],'lr':[],'mAP':[],'mAR':[],'mIoU':[]}
# history = {'train_loss':[],'val_loss':[],'lr':[],'mIoU':[]}

best_mAP = 0

freespace_loss = nn.BCEWithLogitsLoss(reduction='mean')

===========  Optimizer  ==================:
      LR: 0.0001
      step_size: 10
      gamma: 0.9
      num_epochs: 100



In [12]:
# resume="Resnet50_Detection_rand/FFTRadNet_RA_192_56___Apr-20-2024___23:14:26/FFTRadNet_RA_192_56_epoch_detection_89_loss_244328.1924_AP_0.9257_AR_0.6883.pth"

In [13]:
# if resume:
#     print('===========  Resume training  ==================:')
#     dict = torch.load(resume)
#     net.load_state_dict(dict['net_state_dict'])
#     optimizer.load_state_dict(dict['optimizer'])
#     scheduler.load_state_dict(dict['scheduler'])
#     startEpoch = 46
#     history = dict['history']
#     global_step = dict['global_step']
#     print('       ... Start at epoch:', startEpoch)

for epoch in range(startEpoch, 100):

    kbar = pkbar.Kbar(target=len(train_loader), epoch=epoch, num_epochs=num_epochs, width=20, always_stateful=False)

    ###################
    ## Training loop ##
    ###################
    net.train()
    running_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs = data[0].to('cuda').float()
        label_map = data[1].to('cuda').float()
        if (config['model']['SegmentationHead'] == 'True'):
            seg_map_label = data[2].to('cuda').double()

        # reset the gradient
        optimizer.zero_grad()

        # forward pass, enable to track our gradient
        with torch.set_grad_enabled(True):
            outputs = net(inputs)

        classif_loss, reg_loss = pixor_loss(outputs['Detection'], label_map, config['losses'])

        prediction = outputs['Segmentation'].contiguous().flatten()
        label = seg_map_label.contiguous().flatten()
        loss_seg = freespace_loss(prediction, label)
        loss_seg *= inputs.size(0)

        classif_loss *= config['losses']['weight'][0]
        reg_loss *= config['losses']['weight'][1]
        loss_seg *= config['losses']['weight'][2]

        loss = classif_loss + reg_loss + loss_seg

        writer.add_scalar('Loss/train', loss.item(), global_step)
        writer.add_scalar('Loss/train_clc', classif_loss.item(), global_step)
        writer.add_scalar('Loss/train_reg', reg_loss.item(), global_step)
        writer.add_scalar('Loss/train_freespace', loss_seg.item(), global_step)

        # backprop
        loss.backward()
        optimizer.step()

        # statistics
        running_loss += loss.item() * inputs.size(0)

        kbar.update(i, values=[("loss", loss.item()), ("class", classif_loss.item()), ("reg", reg_loss.item()),
                               ("freeSpace", loss_seg.item())])

        global_step += 1

    scheduler.step()

    history['train_loss'].append(running_loss / len(train_loader.dataset))
    history['lr'].append(scheduler.get_last_lr()[0])

    ######################
    ## validation phase ##
    ######################

    eval = run_evaluation(net, val_loader, enc, check_perf=(epoch >= 10),
                          detection_loss=pixor_loss, segmentation_loss=freespace_loss,
                          losses_params=config['losses'])

    history['val_loss'].append(eval['loss'])
    history['mAP'].append(eval['mAP'])
    history['mAR'].append(eval['mAR'])
    history['mIoU'].append(eval['mIoU'])

    kbar.add(1, values=[("val_loss", eval['loss']), ("mAP", eval['mAP']), ("mAR", eval['mAR']),
                        ("mIoU", eval['mIoU'])])

    writer.add_scalar('learning_rate', optimizer.param_groups[0]['lr'], global_step)
    writer.add_scalar('Loss/test', eval['loss'], global_step)
    writer.add_scalar('Metrics/mAP', eval['mAP'], global_step)
    writer.add_scalar('Metrics/mAR', eval['mAR'], global_step)
    writer.add_scalar('Metrics/mIoU', eval['mIoU'], global_step)

    # Saving all checkpoint as the best checkpoint for multi-task is a balance between both --> up to the user to decide
    name_output_file = config['name'] + '_epoch{:02d}_loss_{:.4f}_AP_{:.4f}_AR_{:.4f}_IOU_{:.4f}.pth'.format(epoch,
                                                                                                             eval[
                                                                                                                 'loss'],
                                                                                                             eval[
                                                                                                                 'mAP'],
                                                                                                             eval[
                                                                                                                 'mAR'],
                                                                                                             eval[
                                                                                                                 'mIoU'])
    filename = output_folder / exp_name / name_output_file

    checkpoint = {}
    checkpoint['net_state_dict'] = net.state_dict()
    checkpoint['optimizer'] = optimizer.state_dict()
    checkpoint['scheduler'] = scheduler.state_dict()
    checkpoint['epoch'] = epoch
    checkpoint['history'] = history
    checkpoint['global_step'] = global_step

    torch.save(checkpoint, filename)

    print('')



Epoch: 1/100


RuntimeError: CUDA out of memory. Tried to allocate 576.00 MiB (GPU 0; 10.92 GiB total capacity; 9.40 GiB already allocated; 305.44 MiB free; 9.56 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
if __name__=='__main__':
    # PARSE THE ARGS
    parser = argparse.ArgumentParser(description='FFTRadNet Training')
    parser.add_argument('-c', '--config', default='config/config/config_FFTRadNet_192_56_DetecFalse_LR_0.0001.json',type=str,
                        help='Path to the config file (default: config.json)')
    parser.add_argument('-r', '--resume', default=None, type=str,
                        help='Path to the .pth model checkpoint to resume training')
    args = parser.parse_args()
    config = json.load(open(args.config))
    
    main(config, args.resume)